In [1]:
import gatt
import numpy as np
import time
import datetime


In [2]:
def Write_Bytes(Age ,Height, Gender, c):
    #Wait for respononse from server
   # time.sleep(0.8)
    #Write bytes to initiate communication
    magicBytes = [0xac, 0x02, 0xf7, 0x00, 0x00, 0x00, 0xcc, 0xc3]
    c.write_value(magicBytes)
    #Wait for Write bytes respones
    #time.sleep(1)
    #write 2nd bytes to server
    magicBytes = [0xac, 0x02, 0xfa, 0x00, 0x00, 0x00, 0xcc, 0xc6]
    c.write_value(magicBytes)
    
    #wait for Write bytes respones
    #time.sleep(1)
    #Calculate offset for error checking bits
    Offset = Age + Height - 56
    #Sending Age and height with offset to server
    magicBytes = [0xac, 0x02, 0xfb, 0x01, Age, Height,  0xcc, Offset]
    c.write_value(magicBytes)
    
    #time.sleep(1)
    #Calculate offset for Present date
    now = datetime.datetime.now()
    #Write present date to server(scale)
    Offset = (now.year-1799) + now.month + now.day
    magicBytes = [0xac, 0x02, 0xfd, (now.year - 2000), now.month, now.day, 0xcc, Offset]
    c.write_value(magicBytes)
    
    #time.sleep(1)
    #Calculate offset for time
    now = datetime.datetime.now()
    Offset = now.hour + now.minute
    magicBytes = [0xac, 0x02, 0xfc, now.hour, now.minute, 56, 0xcc, Offset]
    c.write_value(magicBytes)
    
    #time.sleep(1)
    magicBytes = [0xac, 0x02, 0xfe, 0x06, 0x00, 0x00, 0xcc, 0xd0]
    c.write_value(magicBytes)
    
    #time.sleep(0.6)

def body_composition(values):
    #Weight of person shifting higher bit by 8bit position to left and or with lower bit to get 16bit value
    
    weight =  float(((values[0][12] << 8) | values[0][13]) / 10) #. kg
    Fat = float (((values[0][18] << 8 ) | values[0][19])/ 10) #.% 
    Calorie = int((values[1][5] << 8) | values[1][6] ) #. kcal
    bone_mass = float(((values[1][7] << 8 ) | values[1][8]) / 10 ) #. kg
    water = float(((values[1][9] << 8) | values[1][10]) / 10) #.% body composition
    MetabolicAge = int(values[1][11]) #In years

    print ("weight ===================>" + str(weight) +".Kg")
    print ("Fat ======================>" + str(Fat) + ".Kg")
    print ("Calorie ==================>" + str(Calorie) + "Kcal")
    print ("Bone_mass ================>" + str(bone_mass) + "Kg")
    print ("MetabolicAge =============>" + str(MetabolicAge) + "years")
    
    return {"Weight" : weight, 
            "Fat" : Fat, 
            "Calorie" : Calorie, 
            "BoneMass": bone_mass, 
            "Water" : water , 
            "MetabolicAge" : MetabolicAge }
    
    

In [3]:
class AnyDevice(gatt.Device):
    
    def define_variables(self, Age, Weight, Gender, Write):
        self.Age = Age
        self.Weight = Weight
        self.Gender = Gender
        self.Write = Write
        self.values = []
        self.ReadStatus = False
        self.Read = True
        self.count = 0
        #self.manager = manager
        
    def services_resolved(self):
        super().services_resolved()
        for s in self.services:
            if s.uuid == '0000ffb0-0000-1000-8000-00805f9b34fb': #services 0016
                for c in s.characteristics:
                    if (c.uuid == '0000ffb1-0000-1000-8000-00805f9b34fb') and (self.Write == True): #char 0017
                        Write_Bytes(self.Age, self.Weight, self.Gender, c)
                        self.Write = False
                        print(c)
                        
                        #c.enable_notifications()
                    if c.uuid == '0000ffb2-0000-1000-8000-00805f9b34fb' and self.Read == True:
                        c.read_value()
                        c.enable_notifications()
                        print(c)
                        self.Read = False
                    
    def characteristic_value_updated(self, characteristic, value):
        print("Firmware version:", value)
        
        check = value[0] + value[1]       
        if check == 0:
            self.ReadStatus = True
            
        if len(value) == 20 and self.ReadStatus == True :
            self.values.append(value)
            self.count = 1 + self.count

            if self.count == 2:
                manager.stop()
                self.count =0
            
            
        
        
    
        
    
    
        

In [5]:
manager = gatt.DeviceManager(adapter_name='hci0')
device = AnyDevice(manager=manager, mac_address='03:b3:ec:9a:97:b9')
device.define_variables(21,167 ,"male",True) # Age, Height , Gender
device.connect()
manager.run()
body_composition(device.values)


In [8]:
body_composition(device.values)

weight ===================>62.7.Kg
Fat ======================>15.5.Kg
Calorie ==================>1496Kcal
Bone_mass ================>2.6Kg
MetabolicAge =============>16years


{'Weight': 62.7,
 'Fat': 15.5,
 'Calorie': 1496,
 'BoneMass': 2.6,
 'Water': 56.2,
 'MetabolicAge': 16}

ERROR:dbus.connection:Exception in handler for D-Bus signal:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/dbus/connection.py", line 230, in maybe_handle_message
    self._handler(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/gatt/gatt_linux.py", line 539, in properties_changed
    self.service.device.characteristic_value_updated(characteristic=self, value=bytes(value))
  File "<ipython-input-3-31c4c2df465c>", line 39, in characteristic_value_updated
    self.values.append(value)
AttributeError: 'AnyDevice' object has no attribute 'values'
ERROR:dbus.connection:Exception in handler for D-Bus signal:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/dbus/connection.py", line 230, in maybe_handle_message
    self._handler(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/gatt/gatt_linux.py", line 539, in properties_changed
    self.service.device.characteristic_value_updated(characteristic=self, value=byt

Firmware version: b'\x00\x00\x00\x00\x00\x00\x00\x00'
Firmware version: b'\x00\x00\x00\x00\x00\x00\x00\x00'
Firmware version: b'\xac\x02\xfe\x00\x00\x00\xcc\xca'
Firmware version: b'\xac\x02\xfe\x00\x00\x00\xcc\xca'
Firmware version: b'\xac\x02\xfe\x1c\x00\x00\xcc\xe6'
Firmware version: b'\xac\x02\xfe\x1c\x00\x00\xcc\xe6'
Firmware version: b'\xac\x02\xfe\x07\x00\x00\xcc\xd1'
Firmware version: b'\xac\x02\xfe\x07\x00\x00\xcc\xd1'
Firmware version: b'\xac\x02\xfe\x06\xfe\x00\xcc\xce'
Firmware version: b'\xac\x02\xfe\x06\xfe\x00\xcc\xce'
Firmware version: b'\xac\x02\x02\x8d\x00\x00\xce]'
Firmware version: b'\xac\x02\x02\x8d\x00\x00\xce]'
Firmware version: b'\xac\x02\x02\x83\x00\x00\xceS'
Firmware version: b'\xac\x02\x02\x83\x00\x00\xceS'
Firmware version: b'\xac\x02\x02\x83\x00\x00\xceS'
Firmware version: b'\xac\x02\x02\x83\x00\x00\xceS'
Firmware version: b'\xac\x02\x02\x83\x00\x00\xceS'
Firmware version: b'\xac\x02\x02\x83\x00\x00\xceS'
Firmware version: b'\xac\x02\x02\x83\x00\x00\xceS'
F

{'Weight': 64.3,
 'Fat': 17.6,
 'Calorie': 1497,
 'BoneMass': 2.6,
 'Water': 54.4,
 'MetabolicAge': 21}

['z', 'p', 'x', 'k', 'u', 'b', 'c', 'v', 'e', 'w', 'd', 'r', 'o', 'y', 'q', 'f', 's', 'n', 'a', 't', 'g', 'l', '\n', 'm', 'h', 'j', 'i']


In [33]:
x = [1,3,4,5]
y = [3,4,5,6]
c = [x[i] + y [i] for i in range(len(x)) ]
print(c)

[4, 7, 9, 11]


In [9]:
import gatt
import numpy as np
import time
import datetime

class BLE(gatt.Device):
    
    def define_variables(self, Age, Height, Gender, Write, manager):
        self.Age = Age
        self.Height = Height
        self.Gender = Gender
        self.Write = Write
        self.values = []
        self.ReadStatus = False
        self.Read = True
        self.count = 0


    def Write_Bytes(self, c):
        #Write bytes to initiate communication
        magicBytes = [0xac, 0x02, 0xf7, 0x00, 0x00, 0x00, 0xcc, 0xc3]
        c.write_value(magicBytes)

        #write 2nd bytes to server
        magicBytes = [0xac, 0x02, 0xfa, 0x00, 0x00, 0x00, 0xcc, 0xc6]
        c.write_value(magicBytes)
        #Calculate offset for error checking bits
        Offset = self.Age + self.Height - 56
        
        #Sending Age and height with offset to server
        magicBytes = [0xac, 0x02, 0xfb, 0x01, self.Age, self.Height,  0xcc, Offset]
        c.write_value(magicBytes)
        
        #time.sleep(1)
        #Calculate offset for Present date
        now = datetime.datetime.now()
        #Write present date to server(scale)
        Offset = (now.year-1799) + now.month + now.day
        magicBytes = [0xac, 0x02, 0xfd, (now.year - 2000), now.month, now.day, 0xcc, Offset]
        c.write_value(magicBytes)
        
        #time.sleep(1)
        #Calculate offset for time
        now = datetime.datetime.now()
        Offset = now.hour + now.minute
        magicBytes = [0xac, 0x02, 0xfc, now.hour, now.minute, 56, 0xcc, Offset]
        c.write_value(magicBytes)
        
        #time.sleep(1)
        magicBytes = [0xac, 0x02, 0xfe, 0x06, 0x00, 0x00, 0xcc, 0xd0]
        c.write_value(magicBytes)
        
        #time.sleep(0.6)

    def body_composition(self):
        #Weight of person shifting higher bit by 8bit position to left and or with lower bit to get 16bit value
        
        weight =  float(((self.values[0][12] << 8) | self.values[0][13]) / 10) #. kg
        Fat = float (((self.values[0][18] << 8 ) | self.values[0][19])/ 10) #.% 
        Calorie = int((self.values[1][5] << 8) | self.values[1][6] ) #. kcal
        bone_mass = float(((self.values[1][7] << 8 ) | self.values[1][8]) / 10 ) #. kg
        water = float(((self.values[1][9] << 8) | self.values[1][10]) / 10) #.% body composition
        MetabolicAge = int(self.values[1][11]) #In years

        print ("weight ===================>" + str(weight) +".Kg")
        print ("Fat ======================>" + str(Fat) + ".Kg")
        print ("Calorie ==================>" + str(Calorie) + "Kcal")
        print ("Bone_mass ================>" + str(bone_mass) + "Kg")
        print ("MetabolicAge =============>" + str(MetabolicAge) + "years")
        
        return {"Weight" : weight, 
                "Fat" : Fat, 
                "Calorie" : Calorie, 
                "BoneMass": bone_mass, 
                "Water" : water , 
                "MetabolicAge" : MetabolicAge }
                
            
    def services_resolved(self):
        super().services_resolved()
        for s in self.services:
            if s.uuid == '0000ffb0-0000-1000-8000-00805f9b34fb': #services 0016
                for c in s.characteristics:
                    if (c.uuid == '0000ffb1-0000-1000-8000-00805f9b34fb') and (self.Write == True): #char 0017
                        self.Write_Bytes (c)
                        self.Write = False
                        print(c)
                        
                        #c.enable_notifications()
                    if c.uuid == '0000ffb2-0000-1000-8000-00805f9b34fb' and self.Read == True:
                        c.read_value()
                        c.enable_notifications()
                        print(c)
                        self.Read = False


                    
    def characteristic_value_updated(self, characteristic, value):
        print("Firmware version:", value)
        
        check = value[0] + value[1]       
        if check == 0:
            self.ReadStatus = True
            
        if len(value) == 20 and self.ReadStatus == True :
            self.values.append(value)
            self.count = 1 + self.count

            if self.count == 2:
                self.manager.stop()
                self.count =0
            

In [10]:
manager = gatt.DeviceManager(adapter_name='hci0')
device = BLE(manager=manager, mac_address='03:b3:ec:9a:97:b9')
device.define_variables(21,167 ,"male",True, manager) # Age, Height , Gender
device.connect()
manager.run()
device.body_composition()

Firmware version: b'\xac\x02\xfe\x01\x00\x00\xcc\xcb'
Firmware version: b'\xac\x02\xfe\x01\x00\x00\xcc\xcb'
Firmware version: b'\xac\x02\xfe\x01\x00\x00\xcc\xcb'
Firmware version: b'\xac\x02\xf71\x125\xcc;'
Firmware version: b'\xac\x02\xf71\x125\xcc;'
Firmware version: b'\xac\x02\xf71\x125\xcc;'
Firmware version: b'\xac\x02\xfe\t\x00\x00\xcc\xd3\xac\x02\xfe\x07\x00\x00\xcc\xd1'
Firmware version: b'\xac\x02\xfe\t\x00\x00\xcc\xd3\xac\x02\xfe\x07\x00\x00\xcc\xd1'
Firmware version: b'\xac\x02\xfe\t\x00\x00\xcc\xd3\xac\x02\xfe\x07\x00\x00\xcc\xd1'
Firmware version: b'\x00\x00\x00\x00\x00\x00\x00\x00'
Firmware version: b'\x00\x00\x00\x00\x00\x00\x00\x00'
Firmware version: b'\x00\x00\x00\x00\x00\x00\x00\x00'
Firmware version: b'\xac\x02\xfe\x00\x00\x00\xcc\xca'
Firmware version: b'\xac\x02\xfe\x00\x00\x00\xcc\xca'
Firmware version: b'\xac\x02\xfe\x00\x00\x00\xcc\xca'
Firmware version: b'\xac\x02\xfe\x1c\x00\x00\xcc\xe6'
Firmware version: b'\xac\x02\xfe\x1c\x00\x00\xcc\xe6'
Firmware version: b

{'Weight': 61.7,
 'Fat': 16.8,
 'Calorie': 1449,
 'BoneMass': 2.6,
 'Water': 54.6,
 'MetabolicAge': 19}